In [ ]:
pip install semantichar==0.2
pip install pillow==9.2.0
pip install ftfy
pip install einops iopath regex wandb

In [ ]:
!git clone https://github.com/xiyuanzh/SHARE.git

In [ ]:
cd ./SHARE

In [ ]:
mkdir model
mkdir dataset


In [ ]:
!unzip ./easy_imu_phone.zip -d ./

In [2]:
cd /home/keerthiv/HAR_models/share/SHARE/src

/home/keerthiv/HAR_models/share/SHARE/src


/home/keerthiv/.local/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
from torch.nn.utils.rnn import pack_padded_sequence
from torch.cuda.amp import GradScaler, autocast

from semantichar.utils import all_label_augmentation

def DataBatch(data, label, text, l, batchsize, shuffle=True):
    
    n = data.shape[0]
    print( n )
    if shuffle:
        index = np.random.permutation(n)
    else:
        index = np.arange(n)
    for i in range(int(np.ceil(n/batchsize))):
        inds = index[i*batchsize : min(n,(i+1)*batchsize)]
        yield inds, data[inds], label[inds], text[inds], l[inds]
        
def trainer(opt, 
            enc, 
            dec, 
            cross_entropy, 
            optimizer, 
            tr_data, 
            tr_label, 
            tr_text, 
            len_text, 
            break_step, 
            vocab_size, 
            device
):
    """
    Train the model.
    Args:
        opt: user-specified configurations.
        enc: encoder of the model.
        dec: decoder of the model.
        cross_entropy: loss function.
        optimizer: optimizer (default is Adam).
        tr_data, tr_label, tr_text, len_text: training data, label, label sequence, length of the label sequence. 
        break_step: length of the longest label sequence length (i.e., maximum decoding step).
        vocab_size: label name vocabulary size.
        device: cuda or cpu.
    """

    enc.train()
    dec.train()  

    total_loss = 0
    for inds,batch_data, batch_label, batch_text, batch_len in \
        DataBatch(tr_data, tr_label, tr_text, len_text, opt['batchSize'],shuffle=True):
        
        batch_text = all_label_augmentation(batch_text, opt['prob'], break_step, vocab_size)

        batch_data = batch_data.to(device)
        batch_label = batch_label.to(device)
        batch_text = batch_text.to(device)
        batch_len = batch_len.to(device)

        enc_hidden = enc(batch_data)
        pred, batch_text_sorted, decode_lengths, sort_ind \
            = dec(enc_hidden, batch_text, batch_len)
        
        targets = batch_text_sorted[:, 1:]

        pred, *_ = pack_padded_sequence(pred, decode_lengths, batch_first=True)
        targets, *_ = pack_padded_sequence(targets, decode_lengths, batch_first=True)

        loss = cross_entropy(pred, targets.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += len(batch_data) * loss.item()

    total_loss /= len(tr_data)
    
    return total_loss 

  

import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import torch
import torch.nn.functional as F
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import os

def evaluate(opt, 
             enc, 
             dec, 
             test_data, 
             test_label, 
             test_text, 
             test_len_text, 
             pred_dict, 
             seqs, 
             break_step, 
             class_num, 
             vocab_size, 
             device,
             load=True):
    """
    Evaluate the model.
    Args:
        opt: user-specified configurations.
        enc: encoder of the model.
        dec: decoder of the model.
        test_data, test_label, test_text, test_len_text: test data, label, label sequence, length of the label sequence.
        pred_dict: mapping from label token-id sequence to label id.
        seqs: label token-id sequence for all classes.
        break_step: length of the longest label sequence length (i.e., maximum decoding step).
        class_num: number of classes.
        vocab_size: label name vocabulary size.
        device: cuda or cpu.
        load: load saved model weights or not.
    """

    #enc.eval()
    #dec.eval()
    #enc.cpu()
    #dec.cpu()
    #enc.eval()
    #quantized_enc = torch.quantization.convert(enc, inplace=False)
    #print("enc Model converted to quantized version")


    #dec.eval()
    #quantized_dec = torch.quantization.convert(dec, inplace=False)
    #print("dec Model converted to quantized version")

    device = torch.device("cpu")
    #print( opt )
    #print(enc.weight.device)  # Should print 'cuda:0' if on GPU
    #print(dec.weight.device)  # Should print 'cuda:0' if on GPU
    enc.to(device)
    dec.to(device)
    print(next(enc.parameters()).device)
    print(next(dec.parameters()).device)
    print(enc.layer1.weight.device)
    #quantized_enc.eval()
    #quantized_dec.eval()
    #quantized_enc.to(device)
    #quantized_dec.to(device)
    
    if load:
        enc = torch.load(os.path.join('model', f"{opt['run_tag']}_enc.pth"))
        dec = torch.load(os.path.join('model', f"{opt['run_tag']}_dec.pth"))
        #enc.load_state_dict(torch.load(opt['model_path'] + opt['run_tag'] + '_enc.pth', map_location=device, weights_only=False))
        #dec.load_state_dict(torch.load(opt['model_path'] + opt['run_tag'] + '_dec.pth', map_location=device, weights_only=False))

    hypotheses = list()
    batch_size = test_data.size(0)
    pred_whole = torch.zeros_like(test_label)
    seqs = seqs.to(device)

    total_evaluation_time = 0  # Initialize total evaluation time
    total_samples = 0  # Initialize total number of samples

    for batch_idx, (batch_data, batch_label, batch_text, batch_len) in enumerate(
        DataBatch(test_data, test_label, test_text, test_len_text, opt['batchSize'], shuffle=False)
    ):

        batch_data = batch_data.to(device)
        batch_label = batch_label.to(device)
        batch_text = batch_text.to(device)
        batch_len = batch_len.to(device)
        batch_data = batch_data.to_mkldnn()
        enc = enc.to_mkldnn()
        # Start timing after sending to device
        

        batch_size = batch_data.size(0)
        total_samples += batch_size  # Accumulate the number of samples
        start_time = time.time()
        encoder_out = enc(batch_data)  # (batch_size, enc_seq_len, encoder_dim)
        enc_seq_len = encoder_out.size(1)
        encoder_dim = encoder_out.size(2)

        encoder_out = encoder_out.unsqueeze(1).expand(batch_size, class_num, enc_seq_len, encoder_dim)
        encoder_out = encoder_out.reshape(batch_size * class_num, enc_seq_len, encoder_dim)

        k_prev_words = seqs[:, 0].unsqueeze(0).expand(batch_size, class_num).long()  # (batch_size, class_num)
        k_prev_words = k_prev_words.reshape(batch_size * class_num, 1)  # (batch_size * class_num, 1)

        h, c = dec.init_hidden_state(encoder_out)

        seq_scores = torch.zeros((batch_size, class_num)).to(device)

        for step in range(1, break_step):
            embeddings = dec.embedding(k_prev_words).squeeze(1)  # (batch_size * class_num, embed_dim)
            h, c = dec.decode_step(embeddings, (h, c))
            scores = dec.fc(h.reshape(batch_size, class_num, -1))  # (batch_size, class_num, vocab_size)
            scores = F.log_softmax(scores, dim=-1)
            k_prev_words = seqs[:, step].unsqueeze(0).expand(batch_size, class_num).long()
            for batch_i in range(batch_size):
                for class_i in range(class_num):
                    if k_prev_words[batch_i, class_i] != 0:
                        seq_scores[batch_i, class_i] += scores[batch_i, class_i, k_prev_words[batch_i, class_i]]
            k_prev_words = k_prev_words.reshape(batch_size * class_num, 1)  # (batch_size * class_num, 1)

        max_indices = seq_scores.argmax(dim=1)
        for batch_i in range(batch_size):
            max_i = max_indices[batch_i]
            seq = seqs[max_i].tolist()
            hypotheses.append([w for w in seq if w not in {0, vocab_size - 1}])
            pred_whole[batch_i + batch_idx * opt['batchSize']] = pred_dict["#".join(map(str, hypotheses[-1]))]

        # End timing for the batch
        end_time = time.time()
        batch_evaluation_time = end_time - start_time  # Calculate batch evaluation time
        total_evaluation_time += batch_evaluation_time  # Accumulate total evaluation time

        print(f'Batch {batch_idx + 1} Evaluation Time: {batch_evaluation_time:.2f} seconds')

    acc = accuracy_score(test_label.cpu().numpy(), pred_whole.cpu().numpy())
    prec = precision_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)
    rec = recall_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)
    f1 = f1_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)

    print(f'Total Evaluation Time: {total_evaluation_time:.2f} seconds')
    
    # Calculate evaluation time per batch and per sample
    eval_time_per_batch = total_evaluation_time / (batch_idx + 1)
    eval_time_per_sample = total_evaluation_time / total_samples

    print(f'Average Evaluation Time per Batch: {eval_time_per_batch:.2f} seconds')
    print(f'Average Evaluation Time per Sample: {eval_time_per_sample:.6f} seconds')

    return acc, prec, rec, f1




In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.quantization

import semantichar.data 
from semantichar import imagebind_model
from semantichar.imagebind_model import ModalityType

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.quantization import QuantStub, DeQuantStub
import torch.quantization as quant
class CustomBatchNorm1d(nn.Module):
    def __init__(self, num_features):
        super(CustomBatchNorm1d, self).__init__()
        self.bn = nn.BatchNorm1d(num_features)

    def forward(self, x):
        # Convert batch normalization to quantization-aware operations
        if self.training:
            x = self.bn(x)
        else:
            x = torch.nn.functional.batch_norm(
                x, 
                self.bn.running_mean, 
                self.bn.running_var, 
                self.bn.weight, 
                self.bn.bias, 
                training=False
            )
        return x
    

class QuantizedLinear(nn.Module):
    def __init__(self, input_size, output_size, bias=True):
        super(QuantizedLinear, self).__init__()
        self.fc = nn.Linear(input_size, output_size, bias=bias)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)  # Quantize input
        x = self.fc(x)     # Apply linear transformation
        x = self.dequant(x)  # Dequantize output
        return x

    def fuse_model(self):
        # Fuse the linear layer with the quantization stubs
        torch.quantization.fuse_modules(self, ['quant', 'fc', 'dequant'], inplace=True)

    @property
    def weight(self):
        return self.fc.weight

    @property
    def bias(self):
        return self.fc.bias
 
    
class CustomObserver(quant.MinMaxObserver):
    def __init__(self, *args, **kwargs):
        super(CustomObserver, self).__init__(*args, **kwargs)

class QuantizedConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        super(QuantizedConv1d, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=bias)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.weight_fake_quant = quant.FakeQuantize.with_args(observer=quant.default_weight_observer, quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_tensor_affine)()
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)  # Quantize input
        x = self.conv(x)   # Apply convolution
        x = self.bn(x)     # Apply batch normalization
        x = self.relu(x)   # Apply ReLU
        x = self.dequant(x)  # Dequantize output
        return x

    def fuse_model(self):
        # Fuse the conv, batch norm, and ReLU layers
        torch.quantization.fuse_modules(self, [['conv', 'bn', 'relu']], inplace=True)

    @property
    def weight(self):
        return self.conv.weight

    @property
    def bias(self):
        return self.conv.bias

    def apply_weight_fake_quant(self):
        self.weight_fake_quant(self.conv.weight)


class Encoder(nn.Module):

    def __init__(self,
                 d_input: int,
                 d_model: int,
                 d_output: int,
                 seq_len: int):
        super().__init__()

        self.layer1 = nn.Conv1d(in_channels=d_input, out_channels=d_model, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(d_model)
        self.act1 = nn.ReLU()

        self.layer2 = nn.Conv1d(in_channels=d_model, out_channels=d_output, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(d_model)
        self.act2 = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b,t,c = x.size()

        out = self.layer1(x.permute(0,2,1))
        #out = self.act1((out))
        out = self.act1(self.bn1(out))

        out = self.layer2(out)
        #out = self.act2(out)
        out = self.act2(self.bn2(out)) # (b, d_output, seq_len)

        return out.permute(0,2,1) # (b, seq_len, d_output)
        
class Encoder_q(nn.Module):
    def __init__(self, d_input: int, d_model: int, d_output: int, seq_len: int):
        super().__init__()
        self.layer1 = QuantizedConv1d(in_channels=d_input, out_channels=d_model, kernel_size=3, padding=1)
        self.layer2 = QuantizedConv1d(in_channels=d_model, out_channels=d_output, kernel_size=3, padding=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, t, c = x.size()
        out = self.layer1(x.permute(0, 2, 1))
        self.layer1.apply_weight_fake_quant()  # Apply weight fake quantization for layer1
        out = self.layer2(out)
        self.layer2.apply_weight_fake_quant()  # Apply weight fake quantization for layer2
        out = out.permute(0, 2, 1)  # (b, seq_len, d_output)
        return out

class QuantizedLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(QuantizedLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        self.ih = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.hh = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)

        self.quant1 = QuantStub()
        self.dequant1 = DeQuantStub()

        self.quant2 = QuantStub()
        self.dequant2 = DeQuantStub()

        self.quant3 = QuantStub()
        self.dequant3 = DeQuantStub()


    def forward(self, input, hx):
        hx, cx = hx

        # Quantize inputs
        #input = self.quant1(input)
        hx = self.quant2(hx)
        cx = self.quant3(cx)

        # LSTM cell operations
        ih_out = self.ih(input)
        hh_out = self.hh(hx)

        # Dequantize before addition and multiplication
        #ih_out = self.dequant1(ih_out)
        hh_out = self.dequant2(hh_out)
        cx = self.dequant3(cx)

        gates = ih_out + hh_out

        i, f, g, o = gates.chunk(4, 1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        g = torch.tanh(g)
        o = torch.sigmoid(o)

        cy = f * cx + i * g
        hy = o * torch.tanh(cy)

        return hy, cy

    def _init_hidden(self, batch_size, device):
        weight = next(self.parameters()).data
        return (weight.new(batch_size, self.hidden_size).zero_().to(device),
                weight.new(batch_size, self.hidden_size).zero_().to(device))
    
class Decoder(nn.Module):
    
    def __init__(self, embed_dim, decoder_dim, vocab, encoder_dim, device, dropout=0.5):
      
        super(Decoder, self).__init__()

        self.encoder_dim = encoder_dim
        self.embed_dim = embed_dim
        self.decoder_dim = decoder_dim
        self.vocab = vocab
        self.vocab_size = len(vocab)
        self.dropout = dropout
        self.device = device

        self.embedding = nn.Embedding(self.vocab_size, embed_dim)  
        self.dropout = nn.Dropout(p=self.dropout)
        self.decode_step = nn.LSTMCell(embed_dim, decoder_dim, bias=True)  
        self.init_h = nn.Linear(encoder_dim, decoder_dim)  
        self.init_c = nn.Linear(encoder_dim, decoder_dim)  
        self.fc = nn.Linear(decoder_dim, self.vocab_size) 
        self.load_pretrained_embeddings()

    def load_pretrained_embeddings(self):

        inputs = {
            ModalityType.TEXT: semantichar.data.load_and_transform_text(self.vocab, self.device)
        }
        model = imagebind_model.imagebind_huge(pretrained=True)
        model.eval()
        model.to(self.device)
        with torch.no_grad():
            embeddings = model(inputs)['text']
        self.embedding.weight = nn.Parameter(embeddings)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)

    def init_hidden_state(self, encoder_out):
        
        mean_encoder_out = encoder_out.mean(dim=1) 
        h = self.init_h(mean_encoder_out) 
        c = self.init_c(mean_encoder_out)
        return h, c

    def forward(self, encoder_out, encoded_captions, caption_lengths):
        
        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(-1)
        vocab_size = self.vocab_size

        encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  
        seq_len = encoder_out.size(1)

        caption_lengths, sort_ind = caption_lengths.squeeze(1).sort(dim=0, descending=True)
        encoder_out = encoder_out[sort_ind]
        encoded_captions = encoded_captions[sort_ind]

        embeddings = self.embedding(encoded_captions.long()) 

        h, c = self.init_hidden_state(encoder_out)  

        decode_lengths = (caption_lengths - 1).tolist()
        
        predictions = torch.zeros(batch_size, max(decode_lengths), vocab_size).to(self.device)

        for t in range(max(decode_lengths)):
            batch_size_t = sum([l > t for l in decode_lengths]) 
            h, c = self.decode_step(embeddings[:batch_size_t, t, :], \
                (h[:batch_size_t], c[:batch_size_t]))
            preds = self.fc(self.dropout(h))  
            predictions[:batch_size_t, t, :] = preds
        return predictions, encoded_captions, decode_lengths, sort_ind

In [5]:
import argparse
import datetime
import os
import json
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.quantization as quant
from torch.quantization.observer import MovingAverageMinMaxObserver, default_weight_observer

#from semantichar.seq2seq import Encoder, Decoder
#from semantichar.exp import trainer, evaluate
from semantichar.dataset import prepare_dataset

class CustomObserver(MovingAverageMinMaxObserver):
    def calculate_qparams(self):
        scale, _ = super().calculate_qparams()
        zero_point = torch.tensor(0, dtype=torch.int32)
        return scale, zero_point

In [6]:
cd /home/keerthiv/HAR_models/share/SHARE/

/home/keerthiv/HAR_models/share/SHARE


In [7]:
import random
import torch
import numpy as np
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn as nn
import json

# Updated values for the arguments
dataset = 'easy_imu_phone'
data_path = './'
manualSeed = 2023
epochs = 150
early_stopping = 50
batchSize = 16
lr = 1e-4
prob = 0.4
cuda = True
run_tag = 'test'
model_path = './model/'

# Print the updated values
print(f'dataset: {dataset}')
print(f'data_path: {data_path}')
print(f'manualSeed: {manualSeed}')
print(f'epochs: {epochs}')
print(f'early_stopping: {early_stopping}')
print(f'batchSize: {batchSize}')
print(f'lr: {lr}')
print(f'prob: {prob}')
print(f'cuda: {cuda}')
print(f'run_tag: {run_tag}')
print(f'model_path: {model_path}')

# Set random seed
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
np.random.seed(manualSeed)

cudnn.benchmark = True

if torch.cuda.is_available() and not cuda:
    print("You have a cuda device, so you might want to run with --cuda as option")
device = torch.device("cuda:0" if cuda else "cpu")

data_root = data_path + '/dataset/' + dataset
config_file = data_path + '/configs/' + dataset + '.json'
with open(config_file, 'r') as config_file:
    data = json.load(config_file)
    label_dictionary = {int(k): v for k, v in data['label_dictionary'].items()}

tr_data = np.load(data_root + '/x_train.npy')
tr_label = np.load(data_root + '/y_train.npy')

test_data = np.load(data_root + '/x_test.npy')
test_label = np.load(data_root + '/y_test.npy')




dataset: easy_imu_phone
data_path: ./
manualSeed: 2023
epochs: 150
early_stopping: 50
batchSize: 16
lr: 0.0001
prob: 0.4
cuda: True
run_tag: test
model_path: ./model/
Random Seed:  2023


In [8]:
    
seq_len, dim, class_num, vocab_size, break_step, word_list, pred_dict, seqs, \
    tr_data, test_data, \
    tr_label, test_label, \
    tr_text, test_text, \
    len_text, test_len_text = prepare_dataset(tr_data, tr_label, test_data, test_label, label_dictionary)


37
{0: ['start', 'standing', 'end'], 1: ['start', 'walking', 'end'], 2: ['start', 'turning', 'left', 'end'], 3: ['start', 'sitting', 'end'], 4: ['start', 'turning', 'right', 'end'], 5: ['start', 'waving', 'right', 'hand', 'end'], 6: ['start', 'waving', 'left', 'hand', 'end'], 7: ['start', 'jumping', 'end'], 8: ['start', 'lying', 'end'], 9: ['start', 'drinking', 'water', 'end'], 10: ['start', 'throwing', 'end'], 11: ['start', 'kicking', 'right', 'foot', 'end'], 12: ['start', 'kicking', 'left', 'foot', 'end'], 13: ['start', 'golf', 'swinging', 'end'], 14: ['start', 'basketball', 'shooting', 'end'], 15: ['start', 'boxing', 'end'], 16: ['start', 'torso', 'twisting', 'end'], 17: ['start', 'squatting', 'end'], 18: ['start', 'forward', 'bending', 'in', 'standing', 'position', 'end'], 19: ['start', 'forward', 'bending', 'in', 'sitting', 'position', 'end'], 20: ['start', 'leg', 'stretching', 'end'], 21: ['start', 'pushing', 'end'], 22: ['start', 'pulling', 'end'], 23: ['start', 'right', 'hand',

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

class CustomObserver(MovingAverageMinMaxObserver):
    def calculate_qparams(self):
        scale, _ = super().calculate_qparams()
        zero_point = torch.tensor(0, dtype=torch.int32)
        return scale, zero_point
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

enc = Encoder(d_input=dim, d_model=128, d_output=128, seq_len=seq_len).to(device)
dec = Decoder(embed_dim=1024, decoder_dim=128, vocab=word_list, encoder_dim=128, device=device).to(device)

# Apply the quantization configuration to the embedding layers in the decoder
enc.train()
dec.train()

# Move the models to GPU for training
enc.to(device)
dec.to(device)

# Define optimizer and loss function
params = list(enc.parameters()) + list(dec.parameters())
optimizer = optim.Adam(params, lr=1e-4)
cross_entropy = nn.CrossEntropyLoss().to(device)

In [10]:
config = {
    'batchSize': batchSize,
    'epochs': epochs,
    'run_tag': run_tag,
    'dataset': dataset,
    'cuda': cuda,
    'manualSeed': manualSeed,
    'data_path': data_path,
    'early_stopping': early_stopping,
    'lr': 0.0001,
    'prob': prob,
    'model_path': model_path
}


In [11]:
print(seqs)

tensor([[ 0,  1, 36,  0,  0,  0,  0],
        [ 0,  2, 36,  0,  0,  0,  0],
        [ 0,  3,  4, 36,  0,  0,  0],
        [ 0,  5, 36,  0,  0,  0,  0],
        [ 0,  3,  6, 36,  0,  0,  0],
        [ 0,  7,  6,  8, 36,  0,  0],
        [ 0,  7,  4,  8, 36,  0,  0],
        [ 0,  9, 36,  0,  0,  0,  0],
        [ 0, 10, 36,  0,  0,  0,  0],
        [ 0, 11, 12, 36,  0,  0,  0],
        [ 0, 13, 36,  0,  0,  0,  0],
        [ 0, 14,  6, 15, 36,  0,  0],
        [ 0, 14,  4, 15, 36,  0,  0],
        [ 0, 16, 17, 36,  0,  0,  0],
        [ 0, 18, 19, 36,  0,  0,  0],
        [ 0, 20, 36,  0,  0,  0,  0],
        [ 0, 21, 22, 36,  0,  0,  0],
        [ 0, 23, 36,  0,  0,  0,  0],
        [ 0, 24, 25, 26,  1, 27, 36],
        [ 0, 24, 25, 26,  5, 27, 36],
        [ 0, 28, 29, 36,  0,  0,  0],
        [ 0, 30, 36,  0,  0,  0,  0],
        [ 0, 31, 36,  0,  0,  0,  0],
        [ 0,  6,  8, 32, 36,  0,  0],
        [ 0,  4,  8, 32, 36,  0,  0],
        [ 0, 33, 34, 36,  0,  0,  0],
        [ 0,

In [11]:
device = torch.device('cuda')
enc.to(device)
dec.to(device)
tr_data.to(device)
tr_label.to(device)
tr_text.to(device)
len_text.to(device)

for epoch in range(300):
    loss = trainer(
        config, # configs
        enc, # encoder
        dec, # decoder
        cross_entropy, # loss
        optimizer, # optimizer
        tr_data, # training input
        tr_label, # training labels
        tr_text, # training label text sequence
        len_text, # training label text sequence length
        break_step, # max training label text sequence length
        vocab_size, # vocabulary size
        device, # device
    )

    print("epoch: %d total loss: %.4f" % (epoch + 1, loss))


4953
epoch: 1 total loss: 1.7374
4953
epoch: 2 total loss: 0.9214
4953
epoch: 3 total loss: 0.7106
4953
epoch: 4 total loss: 0.6149
4953
epoch: 5 total loss: 0.5310
4953
epoch: 6 total loss: 0.4744
4953
epoch: 7 total loss: 0.4536
4953
epoch: 8 total loss: 0.4004
4953
epoch: 9 total loss: 0.3873
4953
epoch: 10 total loss: 0.3484
4953
epoch: 11 total loss: 0.3269
4953
epoch: 12 total loss: 0.3100
4953
epoch: 13 total loss: 0.2977
4953
epoch: 14 total loss: 0.2781
4953
epoch: 15 total loss: 0.2688
4953
epoch: 16 total loss: 0.2543
4953
epoch: 17 total loss: 0.2457
4953
epoch: 18 total loss: 0.2487
4953
epoch: 19 total loss: 0.2401
4953
epoch: 20 total loss: 0.2215
4953
epoch: 21 total loss: 0.2255
4953
epoch: 22 total loss: 0.2136
4953
epoch: 23 total loss: 0.2105
4953
epoch: 24 total loss: 0.1989
4953
epoch: 25 total loss: 0.1993
4953
epoch: 26 total loss: 0.2006
4953
epoch: 27 total loss: 0.1934
4953
epoch: 28 total loss: 0.1879
4953
epoch: 29 total loss: 0.1857
4953
epoch: 30 total lo

In [12]:
# Save the entire model
torch.save(enc.state_dict(), model_path + run_tag + '_enc.pth')
torch.save(dec.state_dict(), model_path + run_tag + '_dec.pth')

In [12]:
device = torch.device('cpu')

enc = Encoder(d_input=dim, d_model=128, d_output=128, seq_len=seq_len)
dec = Decoder(embed_dim=1024, decoder_dim=128, vocab=word_list, encoder_dim=128, device=device)

# Assuming 'config' is a dictionary containing 'run_tag'
enc_load_path = os.path.join('model', f"{config['run_tag']}_enc.pth")
dec_load_path = os.path.join('model', f"{config['run_tag']}_dec.pth")

# Load the state dictionary from the file
enc_state_dict = torch.load(enc_load_path)
dec_state_dict = torch.load(dec_load_path)

# Load the state dictionary into the model
enc.load_state_dict(enc_state_dict)
dec.load_state_dict(dec_state_dict)

# Ensure the models are on the correct device
enc.to(device)
dec.to(device)
#seqs = torch.load('seqs.pth')

Decoder(
  (embedding): Embedding(37, 1024)
  (dropout): Dropout(p=0.5, inplace=False)
  (decode_step): LSTMCell(1024, 128)
  (init_h): Linear(in_features=128, out_features=128, bias=True)
  (init_c): Linear(in_features=128, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=37, bias=True)
)

In [13]:

#enc.to(device)
#dec.to(device)
enc.eval()
dec.eval()

hypotheses = list()
batch_size = test_data.size(0)
pred_whole = torch.zeros_like(test_label)
seqs = seqs.to(device)
#print(seqs)
config['batchSize'] = 16
total_evaluation_time = 0  # Initialize total evaluation time
total_samples = 0  # Initialize total number of samples
with torch.no_grad():
    for batch_idx, (inds, batch_data, batch_label, batch_text, batch_len) in enumerate(
            DataBatch(test_data, test_label, test_text, test_len_text, config['batchSize'] , shuffle=True)
        ):
            #pred_whole = torch.zeros_like(test_label)
            batch_data = batch_data.to(device)
            batch_label = batch_label.to(device)
            batch_text = batch_text.to(device)
            batch_len = batch_len.to(device)
            #print( batch_idx )
            #print( inds)
            

            batch_size = batch_data.size(0)
            
            total_samples += batch_size  # Accumulate the number of samples
            start_time = time.time()
            #print(enc.layer1.weight.device)
            #print(batch_data.device)
            #print(dec.init_h.weight.device)
            encoder_out = enc(batch_data)  # (batch_size, enc_seq_len, encoder_dim)
            #print(encoder_out.shape)
            enc_seq_len = encoder_out.size(1)
            encoder_dim = encoder_out.size(2)

            encoder_out = encoder_out.unsqueeze(1).expand(batch_size, class_num, enc_seq_len, encoder_dim)
            encoder_out = encoder_out.reshape(batch_size * class_num, enc_seq_len, encoder_dim)
            #print(seqs)
            k_prev_words = seqs[:, 0].unsqueeze(0).expand(batch_size, class_num).long()  # (batch_size, class_num)
            #print(k_prev_words)
            k_prev_words = k_prev_words.reshape(batch_size * class_num, 1)  # (batch_size * class_num, 1)
            
            h, c = dec.init_hidden_state(encoder_out)

            seq_scores = torch.zeros((batch_size, class_num)).to(device)

            for step in range(1, break_step):
                embeddings = dec.embedding(k_prev_words).squeeze(1)  # (batch_size * class_num, embed_dim)
                h, c = dec.decode_step(embeddings, (h, c))
                scores = dec.fc(h.reshape(batch_size, class_num, -1))  # (batch_size, class_num, vocab_size)
                scores = F.log_softmax(scores, dim=-1)
                k_prev_words = seqs[:, step].unsqueeze(0).expand(batch_size, class_num).long()
                for batch_i in range(batch_size):
                    for class_i in range(class_num):
                        if k_prev_words[batch_i, class_i] != 0:
                            seq_scores[batch_i, class_i] += scores[batch_i, class_i, k_prev_words[batch_i, class_i]]
                k_prev_words = k_prev_words.reshape(batch_size * class_num, 1)  # (batch_size * class_num, 1)
            #print( seq_scores )
            max_indices = seq_scores.argmax(dim=1)
            for batch_i in range(batch_size):
                max_i = max_indices[batch_i]
                seq = seqs[max_i].tolist()
                hypotheses.append([w for w in seq if w not in {0, vocab_size - 1}])
                print(batch_i + batch_idx * config['batchSize'])
                #pred_whole[batch_i + batch_idx * config['batchSize']] = pred_dict["#".join(map(str, hypotheses[-1]))]
                #print(batch_i + batch_idx * config['batchSize'])
                #print
                pred_whole[inds[batch_i]] = pred_dict["#".join(map(str, hypotheses[-1]))]
                #print(test_label[inds[batch_i]])
                #print( pred_whole[inds[batch_i]] )
            #print( pred_whole.shape)
            
            end_time = time.time()
            batch_evaluation_time = end_time - start_time  # Calculate batch evaluation time
            total_evaluation_time += batch_evaluation_time  # Accumulate total evaluation time

            #acc = accuracy_score(test_label.cpu().numpy(), pred_whole.cpu().numpy())
            #prec = precision_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)
            #rec = recall_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)
            #f1 = f1_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)

            #print(f'Total Evaluation Time: {total_evaluation_time:.2f} seconds')
    
            # Calculate evaluation time per batch and per sample
            #eval_time_per_batch = total_evaluation_time / (batch_idx + 1)
            #eval_time_per_sample = total_evaluation_time / total_samples

            #print(f'Average Evaluation Time per Batch: {eval_time_per_batch:.2f} seconds')
            #print(f'Average Evaluation Time per Sample: {eval_time_per_sample:.6f} seconds')
            #print('Test Acc: %.4f Macro-Prec: %.4f Macro-Rec: %.4f Macro-F1: %.4f' % (acc, prec, rec, f1))

            #print(f'Batch {batch_idx + 1} Evaluation Time: {batch_evaluation_time:.2f} seconds')
#print( pred_whole.shape )
#print( test_label.shape )

acc = accuracy_score(test_label.cpu().numpy(), pred_whole.cpu().numpy())
prec = precision_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)
rec = recall_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)
f1 = f1_score(test_label.cpu().numpy(), pred_whole.cpu().numpy(), average='macro', zero_division=0)

#print(f'Total Evaluation Time: {total_evaluation_time:.2f} seconds')
    
    # Calculate evaluation time per batch and per sample
#eval_time_per_batch = total_evaluation_time / (batch_idx + 1)
#eval_time_per_sample = total_evaluation_time / total_samples

#print(f'Average Evaluation Time per Batch: {eval_time_per_batch:.2f} seconds')
#print(f'Average Evaluation Time per Sample: {eval_time_per_sample:.6f} seconds')
print('Test Acc: %.4f Macro-Prec: %.4f Macro-Rec: %.4f Macro-F1: %.4f' % (acc, prec, rec, f1))

1320
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2